# Exploración inicial de datos

- Dataset de libros (`books.csv`)
- Ejemplares (`copies(ejemplares).csv`)
- Usuarios (`user_info.csv`)
- Valoraciones (`ratings.csv`)

Objetivos:
- Ver forma y tipos de datos.
- Detectar nulos, duplicados y problemas de calidad.
- Entender cómo se relacionan las claves entre ficheros.


In [6]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("../data/raw")

books_path   = DATA_DIR / "books.csv"
copies_path  = DATA_DIR / "copies(ejemplares).csv"
users_path   = DATA_DIR / "user_info.csv"
ratings_path = DATA_DIR / "ratings.csv"

books_path, copies_path, users_path, ratings_path

(WindowsPath('../data/raw/books.csv'),
 WindowsPath('../data/raw/copies(ejemplares).csv'),
 WindowsPath('../data/raw/user_info.csv'),
 WindowsPath('../data/raw/ratings.csv'))

In [8]:
# Cargar los datos
books = pd.read_csv(books_path, on_bad_lines="skip")
copies = pd.read_csv(copies_path)
users  = pd.read_csv(users_path)
ratings = pd.read_csv(ratings_path)

for name, df in [("books", books), ("copies", copies), ("users", users), ("ratings", ratings)]:
    print(f"{name}: {df.shape[0]} filas, {df.shape[1]} columnas")


books: 9998 filas, 8 columnas
copies: 55327 filas, 2 columnas
users: 501 filas, 4 columnas
ratings: 5976479 filas, 3 columnas


In [10]:
#Ver columnas y tipos
print("BOOKS")
display(books.head())
books.info()

print("\nCOPIES")
display(copies.head())
copies.info()

print("\nUSERS")
display(users.head())
users.info()

print("\nRATINGS")
display(ratings.head())
ratings.info()


BOOKS


,isbn,authors,original_publication_year,original_title,title,language_code,book_id,image_url
0,1554681723,Garth Stein,2006.0,The Art of Racing in the Rain,The Art of Racing in the Rain,eng,216,https://images.gr-assets.com/books/1377206302m...
1,NaN,Rainbow Rowell,2013.0,NaN,Fangirl,eng,324,https://images.gr-assets.com/books/1499565420m...
2,679735771,Bret Easton Ellis,1991.0,American Psycho,American Psycho,eng,499,https://images.gr-assets.com/books/1436934349m...
3,NaN,Jojo Moyes,2015.0,After You,"After You (Me Before You, #2)",eng,566,https://images.gr-assets.com/books/1429029729m...
4,2266079999,Carl Sagan,1985.0,Contact,Contact,eng,1003,https://images.gr-assets.com/books/1408792653m...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9998 entries, 0 to 9997
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   isbn                       9298 non-null   object 
 1   authors                    9998 non-null   object 
 2   original_publication_year  9977 non-null   float64
 3   original_title             9413 non-null   object 
 4   title                      9998 non-null   object 
 5   language_code              8914 non-null   object 
 6   book_id                    9998 non-null   int64  
 7   image_url                  9998 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.0+ KB

COPIES


,copy_id,book_id
0,1,1
1,2,1
2,3,1
3,4,1
4,5,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55327 entries, 0 to 55326
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   copy_id  55327 non-null  int64
 1   book_id  55327 non-null  int64
dtypes: int64(2)
memory usage: 864.6 KB

USERS


,user_id,sexo,comentario,fecha_nacimiento
0,1,Mujer,Me encanta la ficción literaria con profundida...,08/02/1980
1,2,Hombre,"Disfruto la no ficción que me hace pensar, com...",02/04/1990
2,3,Hombre,Prefiero clásicos oscuros como 'Slaughterhouse...,09/11/1993
3,4,Prefiere no responder,Me gustan las historias emocionantes como 'Lif...,12/04/2000
4,5,Prefiere no responder,Adoro los thrillers intensos como 'Brilliance'...,20/02/1987


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           501 non-null    int64 
 1   sexo              501 non-null    object
 2   comentario        501 non-null    object
 3   fecha_nacimiento  501 non-null    object
dtypes: int64(1), object(3)
memory usage: 15.8+ KB

RATINGS


,user_id,copy_id,rating
0,1,11,5
1,1,43,4
2,1,44,5
3,1,56,4
4,1,71,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   copy_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [12]:
print("Duplicados por PK candidata:")

print("books.book_id duplicados:", books.duplicated(subset=["book_id"]).sum())
print("copies.copy_id duplicados:", copies.duplicated(subset=["copy_id"]).sum())
print("users.user_id duplicados:", users.duplicated(subset=["user_id"]).sum())
print("ratings (user_id, copy_id) duplicados:",
      ratings.duplicated(subset=["user_id","copy_id"]).sum())


Duplicados por PK candidata:
books.book_id duplicados: 0
copies.copy_id duplicados: 0
users.user_id duplicados: 0
ratings (user_id, copy_id) duplicados: 0


In [13]:
#Comprobar relaciones entre tablas
# ¿Todos los copies.book_id existen en books.book_id?
missing_books = copies.loc[~copies["book_id"].isin(books["book_id"]), "book_id"].unique()
print("book_id en COPIES sin fila en BOOKS:", missing_books, " (n =", len(missing_books), ")")

# ¿Todos los ratings.copy_id existen en copies.copy_id?
missing_copies = ratings.loc[~ratings["copy_id"].isin(copies["copy_id"]), "copy_id"].unique()
print("copy_id en RATINGS sin fila en COPIES:", len(missing_copies))

# Usuarios: cuántos hay en ratings frente a user_info
ratings_users = set(ratings["user_id"])
info_users    = set(users["user_id"])

print("Usuarios distintos en RATINGS:", len(ratings_users))
print("Usuarios con info demográfica:", len(info_users))
print("Usuarios con info demográfica que tienen ratings:", len(ratings_users & info_users))
print("Usuarios en user_info sin ratings:", info_users - ratings_users)


book_id en COPIES sin fila en BOOKS: [6582 9265]  (n = 2 )
copy_id en RATINGS sin fila en COPIES: 0
Usuarios distintos en RATINGS: 53424
Usuarios con info demográfica: 501
Usuarios con info demográfica que tienen ratings: 500
Usuarios en user_info sin ratings: {999999}


In [14]:
print("Distribución de ratings (1-5):")
display(ratings["rating"].value_counts().sort_index())

print("Ejemplo de comentarios de usuarios:")
display(users.sample(5, random_state=42))


Distribución de ratings (1-5):


rating
1     124195
2     359257
3    1370916
4    2139018
5    1983093
Name: count, dtype: int64

Ejemplo de comentarios de usuarios:


,user_id,sexo,comentario,fecha_nacimiento
362,362,Mujer,"Disfruto distopías como 'The Hunger Games', ro...",20/07/2004
73,74,Prefiere no responder,"Me gustan las distopías como 'Divergent', juve...",22/06/1998
375,375,Hombre,"Amo distopías como 'The Hunger Games', clásico...",07/02/2003
155,156,Hombre,Prefiero juveniles como 'The Fault in Our Star...,17/01/1996
104,105,Hombre,"Disfruto acción como 'The Hunger Games', miste...",15/08/1992


## Resumen de problemas detectados

- **books.csv**
  - 9.998 filas, 8 columnas.
  - Se han tenido que saltar algunas líneas mal formadas (`on_bad_lines="skip"`).
  - `book_id` es único (PK candidata).
  - Hay valores nulos en:
    - `isbn` (~700 filas).
    - `original_publication_year` (~21 filas).
    - `original_title` (~585 filas).
    - `language_code` (~1.084 filas).
  - Faltan al menos 2 `book_id` (p.ej. 6582 y 9265) que sí aparecen en `copies`.

- **copies(ejemplares).csv**
  - 55.327 filas, `copy_id` es único.
  - `book_id` referencia a libros (10.000 distintos).
  - Hay ejemplares que apuntan a `book_id` que no están en `books` (los mismos que faltan allí).

- **user_info.csv**
  - 501 usuarios con información demográfica y comentario.
  - 1 usuario (por ejemplo `999999`) no tiene ratings asociados.

- **ratings.csv**
  - ~5,97 millones de valoraciones.
  - Ratings entre 1 y 5, sin nulos ni valores fuera de rango.
  - Todos los `copy_id` de ratings existen en `copies`.

Conclusión: usaremos `book_id`, `copy_id` y `user_id` como claves principales y tendremos que gestionar:
- Líneas corruptas en `books.csv`.
- Libros sin metadatos completos.
- Diferencia entre usuarios con info demográfica (501) y usuarios totales del sistema (~53k).
